In [1]:
import keras
from keras import layers
from keras import backend as K
from keras.models import Model
import numpy as np
import cv2
import pandas as pd

Using TensorFlow backend.


入力画像の取得と前処理

In [2]:
# 画像サイズ
height = 672
width = 512
# 訓練データの配列化
cloud_train_path_df = pd.read_csv('../data/external/train_img_path_info.csv')
cloud_train_list= []
for i in range(cloud_train_path_df.shape[0]):
    cloud_train_img = cv2.imread('.'+cloud_train_path_df['img_path'][i], 0)
    resize = 4
    desired_im_sz = (512//resize, 672//resize)  # ( height , width )
    cloud_train_img = cv2.resize(cloud_train_img, dsize=desired_im_sz, interpolation=cv2.INTER_CUBIC)
    cloud_train_list.append(cloud_train_img)
cloud_trains = np.array(cloud_train_list)
# テストデータの配列化
cloud_test_path_df = pd.read_csv('../data/external/test_img_path_info.csv')
cloud_test_list= []
for i in range(cloud_test_path_df.shape[0]):
    cloud_test_img = cv2.imread('.'+cloud_test_path_df['img_path'][i], 0)
    resize = 4
    desired_im_sz = (512//resize, 672//resize)  # ( height , width )
    cloud_test_img = cv2.resize(cloud_test_img, dsize=desired_im_sz, interpolation=cv2.INTER_CUBIC)
    cloud_test_list.append(cloud_test_img)
cloud_tests = np.array(cloud_test_list)

モデル構築、学習

In [3]:
# リサイズ後の画像サイズ
rheight = 672//4
rwidth = 512//4 

img_shape = (rheight, rwidth, 1)
batch_size = 16
latent_dim = 2  # Dimensionality of the latent space: a plane

input_img = keras.Input(shape=img_shape)

x = layers.Conv2D(32, 3, padding='same', activation='relu')(input_img)
x = layers.Conv2D(64, 3, padding='same', activation='relu', strides=(2, 2))(x)
x = layers.Conv2D(64, 3, padding='same', activation='relu')(x)
x = layers.Conv2D(64, 3, padding='same', activation='relu')(x)
shape_before_flattening = K.int_shape(x)

x = layers.Flatten()(x)
x = layers.Dense(32, activation='relu')(x)

z_mean = layers.Dense(latent_dim)(x)
z_log_var = layers.Dense(latent_dim)(x)
def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim),
                              mean=0., stddev=1.)
    return z_mean + K.exp(z_log_var) * epsilon

z = layers.Lambda(sampling)([z_mean, z_log_var])
# This is the input where we will feed `z`.
decoder_input = layers.Input(K.int_shape(z)[1:])

# Upsample to the correct number of units
x = layers.Dense(np.prod(shape_before_flattening[1:]),
                 activation='relu')(decoder_input)

# Reshape into an image of the same shape as before our last `Flatten` layer
x = layers.Reshape(shape_before_flattening[1:])(x)

# We then apply then reverse operation to the initial
# stack of convolution layers: a `Conv2DTranspose` layers
# with corresponding parameters.
x = layers.Conv2DTranspose(32, 3, padding='same', activation='relu', strides=(2, 2))(x)
x = layers.Conv2D(1, 3, padding='same', activation='sigmoid')(x)
# We end up with a feature map of the same size as the original input.

# This is our decoder model.
decoder = Model(decoder_input, x)

# We then apply it to `z` to recover the decoded `z`.
z_decoded = decoder(z)
class CustomVariationalLayer(keras.layers.Layer):

    def vae_loss(self, x, z_decoded):
        x = K.flatten(x)
        z_decoded = K.flatten(z_decoded)
        xent_loss = keras.metrics.binary_crossentropy(x, z_decoded)
        kl_loss = -5e-4 * K.mean(
            1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
        return K.mean(xent_loss + kl_loss)

    def call(self, inputs):
        x = inputs[0]
        z_decoded = inputs[1]
        loss = self.vae_loss(x, z_decoded)
        self.add_loss(loss, inputs=inputs)
        # We don't use this output.
        return x

# We call our custom layer on the input and the decoded output,
# to obtain the final model output.
y = CustomVariationalLayer()([input_img, z_decoded])

vae = Model(input_img, y)
vae.compile(optimizer='rmsprop', loss=None)
vae.summary()

x_train = cloud_trains
x_test = cloud_tests

x_train = x_train.astype('float32') / 255.
x_train = x_train.reshape(x_train.shape + (1,))
x_test = x_test.astype('float32') / 255.
x_test = x_test.reshape(x_test.shape + (1,))

vae.fit(x=x_train, y=None,
        shuffle=True,
        epochs=10,
        batch_size=batch_size,
        validation_data=(x_test, None))






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 168, 128, 1)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 168, 128, 32) 320         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 84, 64, 64)   18496       conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_3 (Conv2D)               (None, 84, 64, 64)   36928       conv2d_2[0][0]                   
_________

vaeのモデルと重みを保存

In [4]:
model_json = vae.to_json()
with open('vae.json', 'w') as json_file:
    json_file.write(model_json)
vae.save_weights('vae_cloud_weights.h5')

decoderのモデルと重みを保存

In [5]:
model_json = decoder.to_json()
with open('decoder.json', 'w') as json_file:
    json_file.write(model_json)
decoder.save_weights('decoder_cloud_weights.h5')